In [1]:
!pip install -q pyomo

     |████████████████████████████████| 9.7 MB 2.1 MB/s 
     |████████████████████████████████| 49 kB 4.6 MB/s 


In [2]:
from pyomo.environ import * 

In [3]:
import numpy as np

In [4]:
import pandas as pd

Question1:  Mathematical formulation of given question:
solution: Let us define variables $x_{i,j}  ∀ i,j ∈$ {1,2,3.…8} \\
Here $x_{i,j}$ denotes that $i^{th}$ factory is assigne to $j^{th}$ location.Here value of $x_{i,j}$ =1 if factory is assigned to location otherwise value is zero. \\
let $i^{th}$ factory is assigned to $k^{th}$ location and $j^{th}$ factory is assigned to $m^{th}$ location so now we are to satisfy the demand of $j^{th}$ factory from  $i^{th}$ factory which is $d_{i,j}$ (let) and we are to send this demand from $k^{th}$ location  to  $m^{th}$ location which has cost as $C_{k,m}$ (let) so the objective function is as follows: \\

Minimize$\sum_{i=1}^{8}\sum_{j=1}^{8}\sum_{k=1}^{8}\sum_{m=1}^{8}C_{k,m}d_{i,j}x_{i,k}x_{j,m}$ \\
Since we know that each factory must be alloted at exactly one of these locations so: \\
$\sum_{j=1}^{8}x_{i,j}$=1 ∀ i ∈ {1,2,3,⋯,8} \\
We also know that if each location must be allocated to ecactly one of these factories so: \\
$\sum_{i=1}^{8}x_{i,j}$=1 ∀ j ∈ {1,2,3,⋯,12} \\
$x_{i,j}\geq 0$ ∀ i,j ∈ {1,2,…,8} \\
### Since our objective function is quadratic which is non linear.We will transform it into linear as follows:


# Transformation objective into linear:
Now we define a new variable $y_{i,j,k,l}=x_{i,j}x_{k,l} $ SO now our problem is as follows: \\
Min $\sum_{i,j}^{12,12}\sum_{k,l}^{12,12}C_{i,k}d_{j,l}y_{i,j,k,l} $
s.t.$ \sum_{i}^8y_{i,j,kl} = x_{k,l} j ,k ,l∈\{1,2,⋯,8\}$ \\
$ \sum_{j}^8y_{i,j,kl} = x_{k,l} i ,k ,l∈\{1,2,⋯,8\}$ \\
$y_{i,j,kl}=y_{k,l,i,j} $ \\
$y_{i,j,k,l}\geq 0 \ ∀ \ i,j,k,l∈\{1,2,⋯,8\}$
### Since we have two constraints already:
$\sum_{j=1}^{8}x_{i,j}$=1 ∀ i ∈ {1,2,3,⋯,8} \\
$\sum_{i=1}^{8}x_{i,j}$=1 ∀ j ∈ {1,2,3,⋯,12} \\
$x_{i,j} \geq 0 ∀ \ i,j,k,l ∈  \{1,2,3,⋯,12\} $
## So our new problem is transformed in following manner:
Min $\sum_{i,j}^{12,12}\sum_{k,l}^{12,12}C_{i,k}d_{j,l}y_{i,j,k,l} $
s.t.$ \sum_{i}^8y_{i,j,kl} = x_{k,l} j ,k ,l∈\{1,2,⋯,8\}$ \\
$ \sum_{j}^8y_{i,j,kl} = x_{k,l} i ,k ,l∈\{1,2,⋯,8\}$ \\
$y_{i,j,kl}=y_{k,l,i,j} $ \\
$y_{i,j,k,l}\geq 0 \ ∀ \ i,j,k,l∈\{1,2,⋯,8\}$
$\sum_{j=1}^{8}x_{i,j}$=1 ∀ i ∈ {1,2,3,⋯,8} \\
$\sum_{i=1}^{8}x_{i,j}$=1 ∀ j ∈ {1,2,3,⋯,12} \\
$x_{i,j} \geq 0 \ and \ x_{i,j}\in\{0,1\}∀ \ i,j,k,l ∈  \{1,2,3,⋯,12\} $


### Question 2: If there are  n number of facories and locations are n both then number of variables and constrains:
### Answer: number of variable are $n^4+n^2$ since $x_{1,1}$ denotes factory 1 is assigned to location location 1 similarly we have $n*n$ ;possibilities of each n locations and n factories.And we have $n^4$ possible product of $x_{i,j} and x_{k,l} $ so we have $n^4$ variable of form $y_{i,j,k,l}$ \\
### number of constraints are $n^4+2n^3+2n$.

In [5]:
data_c=np.loadtxt("lab7_ex1_c.txt",dtype='str')

  
        

In [6]:
data_q=np.loadtxt("lab7_ex1_q.txt",dtype='str')

In [7]:
model=ConcreteModel()


In [8]:
row_indices=np.arange(8)
col_indices=np.arange(8)

In [9]:
row_indices

array([0, 1, 2, 3, 4, 5, 6, 7])

In [10]:
model.x=Var(row_indices,col_indices,domain=Binary)
model.y=Var(row_indices,row_indices,row_indices,row_indices,domain=NonNegativeReals)

In [11]:
model.objective=Objective(expr=sum(int(data_c[j+1][l+1])*int(data_q[i+1][k+1])*model.y[i,j,k,l] for i in range(8) for j in range(8) for k in  range(8) for l in range(8)),sense=minimize)

In [12]:
model.constraints=ConstraintList()

In [13]:
for j in range(8):
   for k in range(8):
      for l in range(8):
        model.constraints.add(expr=sum(model.y[i,j,k,l] for i in range(8))==model.x[k,l])
        

In [14]:
for i in range(8):
   for k in range(8):
      for l in range(8):
        model.constraints.add(expr=sum(model.y[i,j,k,l] for j in range(8))==model.x[k,l])

In [15]:
for i in range(8):
  for j in range(8):
    for k in range(8):
      for l in range(8):
        model.constraints.add(expr=model.y[i,j,k,l]==model.y[k,l,i,j])

In [16]:
for i in range(8):
  model.constraints.add(expr=(sum(model.x[i,j] for j in range(8))==1))

In [17]:
for j in range(8):
  model.constraints.add(expr=(sum(model.x[i,j] for i in range(8))==1))

In [18]:
model.pprint()

Streaming output truncated to the last 5000 lines.
         139 :   0.0 : y[0,2,1,2] + y[1,2,1,2] + y[2,2,1,2] + y[3,2,1,2] + y[4,2,1,2] + y[5,2,1,2] + y[6,2,1,2] + y[7,2,1,2] - x[1,2] :   0.0 :   True
         140 :   0.0 : y[0,2,1,3] + y[1,2,1,3] + y[2,2,1,3] + y[3,2,1,3] + y[4,2,1,3] + y[5,2,1,3] + y[6,2,1,3] + y[7,2,1,3] - x[1,3] :   0.0 :   True
         141 :   0.0 : y[0,2,1,4] + y[1,2,1,4] + y[2,2,1,4] + y[3,2,1,4] + y[4,2,1,4] + y[5,2,1,4] + y[6,2,1,4] + y[7,2,1,4] - x[1,4] :   0.0 :   True
         142 :   0.0 : y[0,2,1,5] + y[1,2,1,5] + y[2,2,1,5] + y[3,2,1,5] + y[4,2,1,5] + y[5,2,1,5] + y[6,2,1,5] + y[7,2,1,5] - x[1,5] :   0.0 :   True
         143 :   0.0 : y[0,2,1,6] + y[1,2,1,6] + y[2,2,1,6] + y[3,2,1,6] + y[4,2,1,6] + y[5,2,1,6] + y[6,2,1,6] + y[7,2,1,6] - x[1,6] :   0.0 :   True
         144 :   0.0 : y[0,2,1,7] + y[1,2,1,7] + y[2,2,1,7] + y[3,2,1,7] + y[4,2,1,7] + y[5,2,1,7] + y[6,2,1,7] + y[7,2,1,7] - x[1,7] :   0.0 :   True
         145 :   0.0 : y[0,2,2,0] + y[1,2,2

In [19]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 159447 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [20]:
opt_cbc = SolverFactory('cbc')

In [21]:
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver time taken',result.solver.time)

Solver status: ok
Solver termination condition: optimal
Solver time taken 584.422994852066


In [22]:
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in row_indices:
  for j in col_indices:
    if model.x[i,j].value==1:
      print(f'Factory {i+1} is assinged to location:{j+1}' )


Objective =  14889.0

Decision Variables
Factory 1 is assinged to location:2
Factory 2 is assinged to location:8
Factory 3 is assinged to location:7
Factory 4 is assinged to location:6
Factory 5 is assinged to location:1
Factory 6 is assinged to location:3
Factory 7 is assinged to location:5
Factory 8 is assinged to location:4


## Q3: Solve the problem and report which facility must be opened at each location.
### Optimal cost: 14889.0
Factory 1 is assinged to location:2 \\
Factory 2 is assinged to location:8 \\
Factory 3 is assinged to location:7 \\
Factory 4 is assinged to location:6 \\
Factory 5 is assinged to location:1 \\
Factory 6 is assinged to location:3 \\
Factory 7 is assinged to location:5 \\
Factory 8 is assinged to location:4 \\

### Q4:
 Now change the integer variables in your model to continuous variables, and re-solve the problem. Report the solution (only the non-zero values of the solution)
# sol:
now we change the domain of $x_{i,j}$ to non negative reals from binary.

In [23]:
model.x.domain=Reals
model.x.setub(1)
model.x.setlb(0)

In [24]:
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver time taken',result.solver.time)

Solver status: ok
Solver termination condition: optimal
Solver time taken 1.498506784439087


In [26]:
print('\nObjective = ', model.objective())

print('\nDecision Variables')
for i in row_indices:
  for j in col_indices:
    if model.x[i,j].value==1:
      print(f'Factory {i+1} is assinged to location:{j+1}' )
    else:
      None
    if model.x[i,j].value!=0:
      print(f'* x[{i}{j}]:',model.x[i,j].value)   


Objective =  11790.875010123458

Decision Variables
* x[00]: 0.084983499
* x[01]: 0.33367696
* x[02]: 0.048316653
* x[03]: 0.10039981
* x[04]: 0.051444116
* x[05]: 0.25047975
* x[06]: 0.066562267
* x[07]: 0.064136939
* x[10]: 0.19073548
* x[11]: 0.026261001
* x[12]: 0.11864323
* x[13]: 0.1917243
* x[14]: 0.17399296
* x[15]: 0.022830959
* x[16]: 0.15180168
* x[17]: 0.1240104
* x[20]: 0.10388789
* x[22]: 0.17341885
* x[23]: 0.20991576
* x[24]: 0.15765049
* x[25]: 0.12777006
* x[26]: 0.12970289
* x[27]: 0.09765406
* x[30]: 0.07739104
* x[31]: 0.0017389244
* x[32]: 0.16883651
* x[33]: 0.041916945
* x[34]: 0.22827876
* x[35]: 0.13528148
* x[36]: 0.11864323
* x[37]: 0.22791312
* x[40]: 0.20537663
* x[41]: 0.14760491
* x[42]: 0.15701317
* x[43]: 0.043650923
* x[44]: 0.038651246
* x[45]: 0.0772711
* x[46]: 0.15673067
* x[47]: 0.17370134
* x[50]: 0.022830959
* x[52]: 0.19583209
* x[53]: 0.019085987
* x[54]: 0.26693
* x[55]: 0.32892201
* x[56]: 0.06176476
* x[57]: 0.1046342
* x[60]: 0.063140624

# Sol3
After changing domain of variable we get:
objective function:11790.875010123458 \\
Since according to our assumptions we know that if value of variable $x_{i,j}$ is 1 that means factory i is assigned to location j but there is no such  variable $x_{i,j}$ which has value 1 so in this method we can't justify that which factory is assigned to which location.
value of non zero varaibles:
Decision Variables
* x[00]: 0.084983499
* x[01]: 0.33367696
* x[02]: 0.048316653
* x[03]: 0.10039981
* x[04]: 0.051444116
* x[05]: 0.25047975
* x[06]: 0.066562267
* x[07]: 0.064136939
* x[10]: 0.19073548
* x[11]: 0.026261001
* x[12]: 0.11864323
* x[13]: 0.1917243
* x[14]: 0.17399296
* x[15]: 0.022830959
* x[16]: 0.15180168
* x[17]: 0.1240104
* x[20]: 0.10388789
* x[22]: 0.17341885
* x[23]: 0.20991576
* x[24]: 0.15765049
* x[25]: 0.12777006
* x[26]: 0.12970289
* x[27]: 0.09765406
* x[30]: 0.07739104
* x[31]: 0.0017389244
* x[32]: 0.16883651
* x[33]: 0.041916945
* x[34]: 0.22827876
* x[35]: 0.13528148
* x[36]: 0.11864323
* x[37]: 0.22791312
* x[40]: 0.20537663
* x[41]: 0.14760491
* x[42]: 0.15701317
* x[43]: 0.043650923
* x[44]: 0.038651246
* x[45]: 0.0772711
* x[46]: 0.15673067
* x[47]: 0.17370134
* x[50]: 0.022830959
* x[52]: 0.19583209
* x[53]: 0.019085987
* x[54]: 0.26693
* x[55]: 0.32892201
* x[56]: 0.06176476
* x[57]: 0.1046342
* x[60]: 0.063140624
* x[61]: 0.31854682
* x[62]: 0.06176476
* x[63]: 0.26544384
* x[64]: 0.083052425
* x[65]: 0.023820054
* x[66]: 0.063140624
* x[67]: 0.12109086
* x[70]: 0.25165388
* x[71]: 0.17217138
* x[72]: 0.076174743
* x[73]: 0.12786244
* x[75]: 0.033624599
* x[76]: 0.25165388
* x[77]: 0.086859086

# Question4: 
### Are the optimal costs for both problems same? Are the values of the variables still integer-valued? If yes, explain why.
## Answer:
No the optimal cost for both problem is not same.
All the variables are not integervalued since value of $x_{21},x_{51},x_{74}=0$ and others are not integers.So we can't say that all variables are integervalued.
We get this because we got optimal solution on these values.